We will use [Curve P-384](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.186-4.pdf)  as an example:

$p = 2^{384} - 2^{128} - 2^{96} + 2^{32} - 1$

Points satisfy the following equation:

$y^2 = x^3 + Ax + B \mod p$

with $A$ and $B$ as below. 

In [1]:
p = 2**384 - 2**128 - 2**96 + 2**32 - 1
Fp = FiniteField(p)
A = -3
B = 0xb3312fa7e23ee7e4988e056be3f82d19181d9c6efe8141120314088f5013875ac656398d8a2ed19d2a85c8edd3ec2aef
E = EllipticCurve(Fp,[A,B])
print(E)

Elliptic Curve defined by y^2 = x^3 + 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112316*x + 27580193559959705877849011840389048093056905856361568521428707301988689241309860865136260764883745107765439761230575 over Finite Field of size 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112319


The standard also defines the generator point G via two coordinates Gx and Gy:

In [2]:
Gx = 0xaa87ca22be8b05378eb1c71ef320ad746e1d3b628ba79b9859f741e082542a385502f25dbf55296c3a545e3872760ab7
Gy = 0x3617de4a96262c6f5d9e98bf9292dc29f8f41dbd289a147ce9da3113b5f0b8c00a60b1ce1d7e819d7a431d7c90ea0e5f
assert(E.is_on_curve(Gx,Gy))
G = E([Gx,Gy])
print(G)

(26247035095799689268623156744566981891852923491109213387815615900925518854738050089022388053975719786650872476732087 : 8325710961489029985546751289520108179287853048861315594709205902480503199884419224438643760392947333078086511627871 : 1)


We can now generate random scalars and compute the exponentiation in the group.

In [3]:
x = ZZ(Fp.random_element())
X = x*G
print(X)
y = ZZ(Fp.random_element())
Y = y*G
print(Y)

(8430767636021080051196762888133599581365188839664834865942849537428426363829165324570769546835566866393507313348952 : 23505202033048537628103831509319056592600000158527892871153306525021416029848069956348209251332936383586271933052011 : 1)
(16636008035956144572510450552700890558453801285568366289556139680157619374565613178188519286001988686848621675709535 : 36142904930032503032746191427580161725381447955271405358531713719231092000952904653289662546195211123878747848621265 : 1)


And EC Diffie-Hellman protocol works

In [4]:
assert(x*Y == y*X)

In [5]:
def GenPubKey(A, B, p, xP, yP):
    P = (xP, yP)
    nA = 3 # how to chose this secret multiplier? Sample from eliptic curve?
    Qa = nA*P
    
    return (Qa, nA)

def Encript(A, B, p, xP, yP, Qa, m1, m2):
    P = (xP, yP)
    k = 3 # from where, at random???
    S = k*Qa
    xS = S[0]
    yS = S[1]

    
    R = k*P
    c1 = (xS*m1) % p
    c2 = (yS*m2) % p
    
    return (R, c1, c2)

def Decript(A, B, p, xP, yP, R, c1, c2):
    P = (xP, yP)
    #Qa = nA*P
    
    k = R/P
    #S = k*Qa
    xS = S[0]
    yS = S[1]
    
    #c1 = (xS*m1) % p
    #c2 = (yS*m2) % p
    
    #(a + x) mod m = b  
    #a + x = nm + b  
    #x = nm + b - a for some integer n
    
    m1 = 
    
    return (m1, m2)

In [7]:
print(GenPubKey(A, B, 2, 3, 4))

((3, 4, 3, 4, 3, 4), 3)
